# get ann

In [3]:
from cmschina_tianyan.session import CmsSession
import pandas as pd
dataset = CmsSession.init(appId="cb632213cf764a019b32f8cc9d2040b7", appSecret="b23700b9fbf0090709c7b72ac0fdcb390cd19cc663bcf1c7a07c12b8c32bf377")

stk_code_list = ['300587.SZ',
'603185.SH',
'600674.SH',
'002203.SZ',
'601127.SH',
'002597.SZ',
'603348.SH',
'300487.SZ',
'603876.SH',
'601677.SH',
'300655.SZ',
'002078.SZ',
'002738.SZ', #2022-10-14新增4个对子
'002812.SZ',
'603596.SH',
'300332.SZ',]

# stk_code = '603355.SH'
df_ann_res = []
for stk_code in stk_code_list:
    sql = f''' 
    SELECT * FROM wind_admin.ASHAREANNINF 
    where S_INFO_WINDCODE='{stk_code}' 
    order by ANN_DT desc
    '''

    df_ann = dataset.get_data(sql)
    df_ann_res.append(df_ann)
    # df_ann.to_excel(f'./ann/{stk_code}.xlsx')
df_total_ann = pd.concat(df_ann_res)
df_total_ann['serial'] = [i for i in range(len(df_total_ann))]
df_total_ann['author'] = ['上市公司']*len(df_total_ann)
df_total_ann = df_total_ann.set_index('serial')
df_total_ann = df_total_ann[['S_INFO_WINDCODE','ANN_DT','N_INFO_TITLE','author']]
cols = ['code','date','title','author']
df_total_ann.columns = cols
df_total_ann    

[2022-11-03T14:52:31.441024+0800] [56965] [INFO] [session.py] [63] PROD Env Init Completed!
[2022-11-03T14:52:33.212285+0800] [56965] [DEBUG] [tianyan_provider.py] [130] retrieve total 1272 rows of data
[2022-11-03T14:52:34.765509+0800] [56965] [DEBUG] [tianyan_provider.py] [130] retrieve total 885 rows of data
[2022-11-03T14:52:37.463210+0800] [56965] [DEBUG] [tianyan_provider.py] [130] retrieve total 2350 rows of data
[2022-11-03T14:52:38.691716+0800] [56965] [DEBUG] [tianyan_provider.py] [130] retrieve total 2107 rows of data
[2022-11-03T14:52:40.012761+0800] [56965] [DEBUG] [tianyan_provider.py] [130] retrieve total 1565 rows of data
[2022-11-03T14:52:42.227762+0800] [56965] [DEBUG] [tianyan_provider.py] [130] retrieve total 1577 rows of data
[2022-11-03T14:52:43.690364+0800] [56965] [DEBUG] [tianyan_provider.py] [130] retrieve total 735 rows of data
[2022-11-03T14:52:44.943380+0800] [56965] [DEBUG] [tianyan_provider.py] [130] retrieve total 1315 rows of data
[2022-11-03T14:52:46.5

,code,date,title,author
serial,,,,
0,300587.SZ,2022-10-28,天铁股份:浙江天铁实业股份有限公司关于使用部分闲置募集资金购买理财产品到期赎回的公告,上市公司
1,300587.SZ,2022-10-26,天铁股份:浙江天铁实业股份有限公司第四届董事会第二十七次会议决议公告,上市公司
2,300587.SZ,2022-10-26,天铁股份:浙江天铁实业股份有限公司2022年第三季度报告披露的提示性公告,上市公司
3,300587.SZ,2022-10-26,天铁股份:浙江天铁实业股份有限公司关于公开发行可转换公司债券募集资金投资项目结项的公告,上市公司
4,300587.SZ,2022-10-26,天铁股份:2022年三季度报告,上市公司
...,...,...,...,...
23282,300332.SZ,2012-06-07,天壕节能:第一届董事会第十三次会议决议,上市公司
23283,300332.SZ,2012-06-07,天壕节能:第一届董事会第三次会议决议,上市公司
23284,300332.SZ,2012-03-05,天壕节能:发行人控股股东、实际控制人对招股说明书的确认意见,上市公司


# get news

In [4]:
import pandas as pd
import requests
import random
import time
import os
import openpyxl
from bs4 import BeautifulSoup
def get_url(code,pages):
    '''
    获取东方财富网股吧链接列表
    code是指公司代码
    page是值爬取页数
    '''
    url_list = []
    for page in range(1,pages+1):
        url = f"http://guba.eastmoney.com/list,{code},1,f_{page}.html"
        url_list.append(url)
        
    return url_list

def get_news(url_list,code):
    '''
    获取东方财富网新闻列表至本地xls
    url_list是指链接列表
    '''
    headers = {
        # 'User-Agent': UserAgent(verify_ssl=False).random,
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
        'cookie': 'qgqp_b_id=b90ae58b50ba4b8868c1988b80823e4c; st_si=62771611099798; st_asi=delete; st_pvi=50748890314615; st_sp=2022-10-24 11:22:01; st_inirUrl=http://guba.eastmoney.com/list,hk01810,1,f_1.html; st_sn=2; st_psi=20221024112234183-117001300541-9748756866'
    }
    
    # 保存爬取内容
    outwb = openpyxl.Workbook() # 打开一个将写的文件
    outws = outwb.create_sheet(index=0) # 在将写的文件创建sheet
    outws.cell(row = 1, column = 1, value = "read")
    outws.cell(row = 1, column = 2, value = "comment")
    outws.cell(row = 1, column = 3, value = "title")
    outws.cell(row = 1, column = 4, value = "author")
    outws.cell(row = 1, column = 5, value = "renew")
    outws.cell(row = 1, column = 6, value = "link")
    index = 2
    
    for i in range(len(url_list)):
        url = url_list[i]
        res = requests.get(url,headers = headers)
        res.encoding = res.apparent_encoding
        html = res.text
        soup = BeautifulSoup(html,"html.parser")
        read_list = soup.select(".l1.a1")[1:]
        comment_list = soup.select(".l2.a2")[1:]
        title_list = soup.select(".l3.a3")[1:]
        author_list = soup.select(".l4.a4")[1:]
        renew_list = soup.select(".l5.a5")[1:]
        for k in range(len(title_list)):
            outws.cell(row = index, column = 1, value = str(read_list[k].text.strip()))
            outws.cell(row = index, column = 2, value = str(comment_list[k].text.strip()))
            outws.cell(row = index, column = 3, value = str(title_list[k].select('a')[0]["title"]))
            outws.cell(row = index, column = 4, value = str(author_list[k].text.strip()))
            outws.cell(row = index, column = 5, value = str(renew_list[k].text.strip()))
            outws.cell(row = index, column = 6, value = str(title_list[k].select('a')[0]["href"]))                                                
            index += 1
            # print(str(title_list[k].select('a')[0]["href"]))
            # print(title_list[k].select('a')[0]["title"],renew_list[k].text.strip())
        time.sleep(random.uniform(3,4))
    save_path = f"./news_ann/news/tmp/{code}.xlsx"            
    outwb.save(save_path)
    df = pd.read_excel(save_path)
    return df 
 

stk_code_list = ['300587.SZ',
'603185.SH',
'600674.SH',
'002203.SZ',
'601127.SH',
'002597.SZ',
'603348.SH',
'300487.SZ',
'603876.SH',
'601677.SH',
'300655.SZ',
'002078.SZ',
'002738.SZ', #2022-10-14新增4个对子
'002812.SZ',
'603596.SH',
'300332.SZ',]
df_news_res = []
for stk_code in stk_code_list:
    code = stk_code.split('.')[0]
    pages = 1
    url_list = get_url(code,pages)
    df_tmp = get_news(url_list,code)
    df_tmp['code'] = [stk_code]*len(df_tmp)
    df_news_res.append(df_tmp)
    print(f"{code} 运行完成")    
df_total_news = pd.concat(df_news_res)

300587 运行完成
603185 运行完成
600674 运行完成
002203 运行完成
601127 运行完成
002597 运行完成
603348 运行完成
300487 运行完成
603876 运行完成
601677 运行完成
300655 运行完成
002078 运行完成
002738 运行完成
002812 运行完成
603596 运行完成
300332 运行完成


In [5]:
df_total_news = pd.concat(df_news_res)
df_total_news['serial'] = [i for i in range(len(df_total_news))]
df_total_news = df_total_news.set_index('serial')

df_total_news['date'] = df_total_news['renew'].apply(lambda x:'2022-' + x.split(' ')[0])
cols = ['code','date','title','author']
df_total_news = df_total_news[cols]
df_total_news

,code,date,title,author
serial,,,,
0,300587.SZ,2022-11-03,天铁股份：融资净买入1866.18万元，融资余额1.13亿元（11-02）,天铁股份资讯
1,300587.SZ,2022-11-03,天铁股份11月02日被深股通减持70.02万股,天铁股份资讯
2,300587.SZ,2022-11-02,天铁股份：融资净买入750.88万元，融资余额9454.39万元（11-01）,天铁股份资讯
3,300587.SZ,2022-11-02,天铁股份11月01日获深股通增持146.09万股,天铁股份资讯
4,300587.SZ,2022-11-01,天铁股份：融资净买入1850.62万元，融资余额8703.51万元（10-31）,天铁股份资讯
...,...,...,...,...
1275,300332.SZ,2022-09-03,天壕环境09月02日获深股通增持7.81万股,天壕环境资讯
1276,300332.SZ,2022-09-02,天壕环境09月01日被深股通减持180.1万股,天壕环境资讯
1277,300332.SZ,2022-09-01,天壕环境08月31日获深股通增持178.99万股,天壕环境资讯


# merge

In [6]:
df_total_news_ann = pd.concat([df_total_ann,df_total_news])
df_total_news_ann['serial'] = [i for i in range(len(df_total_news_ann))]
df_total_news_ann = df_total_news_ann.set_index('serial')


In [7]:
from WindPy import *
#Wind API 使用W账户密码登录
w.start() 
#return True and then can go to next operation.
w.isconnected()

22.9.1.52437
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [8]:
df_stk_name = w.wss('300587.SZ,603185.SH,600674.SH,002203.SZ,601127.SH,002597.SZ,603348.SH,300487.SZ,603876.SH,601677.SH,300655.SZ,002078.SZ,002738.SZ, 002812.SZ,603596.SH,300332.SZ', "sec_name", "",usedf=True)[1]
stk_name_dic = df_stk_name.to_dict()['SEC_NAME']
# df_stk_name
df_total_news_ann['name'] =  df_total_news_ann['code'].map(stk_name_dic)
ordered_cols = ['code','name','date','title','author']

df_total_news_ann = df_total_news_ann [ordered_cols]
df_total_news_ann['code_name'] = df_total_news_ann['code'] + df_total_news_ann['name']
df_total_news_ann

,code,name,date,title,author,code_name
serial,,,,,,
0,300587.SZ,天铁股份,2022-10-28,天铁股份:浙江天铁实业股份有限公司关于使用部分闲置募集资金购买理财产品到期赎回的公告,上市公司,300587.SZ天铁股份
1,300587.SZ,天铁股份,2022-10-26,天铁股份:浙江天铁实业股份有限公司第四届董事会第二十七次会议决议公告,上市公司,300587.SZ天铁股份
2,300587.SZ,天铁股份,2022-10-26,天铁股份:浙江天铁实业股份有限公司2022年第三季度报告披露的提示性公告,上市公司,300587.SZ天铁股份
3,300587.SZ,天铁股份,2022-10-26,天铁股份:浙江天铁实业股份有限公司关于公开发行可转换公司债券募集资金投资项目结项的公告,上市公司,300587.SZ天铁股份
4,300587.SZ,天铁股份,2022-10-26,天铁股份:2022年三季度报告,上市公司,300587.SZ天铁股份
...,...,...,...,...,...,...
24562,300332.SZ,天壕环境,2022-09-03,天壕环境09月02日获深股通增持7.81万股,天壕环境资讯,300332.SZ天壕环境
24563,300332.SZ,天壕环境,2022-09-02,天壕环境09月01日被深股通减持180.1万股,天壕环境资讯,300332.SZ天壕环境
24564,300332.SZ,天壕环境,2022-09-01,天壕环境08月31日获深股通增持178.99万股,天壕环境资讯,300332.SZ天壕环境


# output to excel

In [10]:
import openpyxl 
import xlwings as xw
wb_file = './news_ann/total_news_ann/news_ann.xlsx'
book = openpyxl.load_workbook(wb_file)   #读取你要写入的workbook
#和pd.read_excel() 用于将Dataframe写入excel。xls用xlwt。xlsx用openpyxl
writer = pd.ExcelWriter(wb_file, engine='openpyxl')   
##此时的writer里还只是读写器. 然后将上面读取的book复制给writer  
writer.book = book
#转化为字典的形式
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
#将data写入writer
df_total_news_ann.to_excel(writer,sheet_name="origin table",index=False)
writer.save()

In [9]:
# df_total_news_ann.to_excel('./news_ann/total_news_ann/news_ann.xlsx')